In [8]:
import os
tracing = os.environ.get("LANGSMITH_TRACING")
api_key = os.environ.get("LANGSMITH_API_KEY")
endpoint = os.environ.get("LANGSMITH_ENDPOINT")
project = os.environ.get("LANGSMITH_PROJECT")

print(tracing, api_key, endpoint, project)

true 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 https://api.smith.langchain.com CV_Filter


In [ ]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "CV_Filter"


In [10]:
from langsmith import utils
utils.tracing_is_enabled()

True

In [ ]:
import re
from collections import defaultdict
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

# === SECTION PARSING UTILITIES ===
SECTION_KEYWORDS = {
    "summary": r"^\s*(summary|profile|objective|about me|professional summary)\s*:?\s*$",
    "experience": r"^\s*(experience|work history|employment|professional experience)\s*:?\s*$",
    "education": r"^\s*(education|academic background|qualifications)\s*:?\s*$",
    "skills": r"^\s*(skills|technical skills|competencies|proficiencies|technical expertise)\s*:?\s*$",
    "certifications": r"^\s*(certifications|licenses|courses|training)\s*:?\s*$",
    "projects": r"^\s*(projects|personal projects|key projects)\s*:?\s*$",
    "languages": r"^\s*(languages)\s*:?\s*$",
}

DEFAULT_SECTION = "header"
LIST_SECTIONS = ["experience", "education", "certifications", "projects"]

def identify_section(line):
    for section, pattern in SECTION_KEYWORDS.items():
        if re.match(pattern, line, re.IGNORECASE):
            return section
    return None

def clean_text(text):
    if not text:
        return ""
    text = re.sub(r'[ \t]+', ' ', text)
    lines = [line.strip() for line in text.strip().split('\n')]
    return "\n".join([line for line in lines if line])

def naive_split_entries(text_block):
    if not text_block:
        return []
    entries = re.split(r'\n\s*\n+', text_block.strip())
    return [entry.strip() for entry in entries if entry.strip()]

def chunk_text_into_sections(text):
    lines = text.split('\n')
    sections_content = defaultdict(str)
    current_section = DEFAULT_SECTION

    for line in lines:
        stripped = line.strip()
        matched_section = identify_section(stripped)
        if matched_section:
            current_section = matched_section
            continue
        sections_content[current_section] += line + "\n"

    chunked_sections = {}
    for section_name, content in sections_content.items():
        if section_name in LIST_SECTIONS:
            chunked_sections[section_name] = naive_split_entries(clean_text(content))
        else:
            chunked_sections[section_name] = clean_text(content)

    return chunked_sections

# === LOADER SETUP WITH LANGCHAIN ===
def pdf_loader_single(file_path):
    return PyPDFLoader(file_path, mode="single")

loader = DirectoryLoader(
    r"D:\GBG\tasks\langchain\last\cv_pdfs",
    glob="**/*.pdf",
    loader_cls=pdf_loader_single
)

docs = loader.load()
print(f"Loaded {len(docs)} documents")

# === PROCESS DOCUMENT TEXTS ===
structured_docs = []
for doc in docs:
    text = doc.page_content
    pdf_path = doc.metadata["source"]
    structured = chunk_text_into_sections(text)
    structured_docs.append({
        "file": pdf_path,
        "sections": structured
    })

# === DISPLAY OUTPUT ===
for doc in structured_docs:
    print(f"\nFile: {doc['file']}")
    for section, content in doc["sections"].items():
        print(f"\n== {section.upper()} ==")
        if isinstance(content, list):
            for entry in content:
                print(f"- {entry}")
        else:
            print(content)


In [ ]:
from langchain_openai import AzureOpenAIEmbeddings ,AzureChatOpenAI
#Embedding_Model
embedding_model = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint="",
    api_key="",
    openai_api_version="2023-05-15",
)

#Chat Model
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",
    azure_endpoint="",
    api_key="",
    openai_api_version="2023-05-15",
    temperature=0
)

In [ ]:
# llm.invoke("Hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BVGmkpfR5f2IPuJEtMUScuyzw5m2M', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run--4c465f09-2309-4e65-a2a6-d689dd269cec-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
##Qdrant db
!pip install qdrant-client[fastembed]

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.6 MB 4.2 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.6 MB 4.0 MB/s eta 0:00:01
   -------------------------------- ------- 2.1/2.6 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   -- ------------------------------------- 0.8/14.5 MB 3.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.6/14.5 MB 3.8 MB/s eta 0:00:04
   ------ --------------------------------- 2.4/14.5 MB 3.8 MB/s eta 0:00:04
   -------- ------------------------------- 3.1/14.5 MB 3.9 MB/s eta 0:00:03
   ---------- ----------------------------- 3.7/14.5 MB 3.9 MB/s eta 0:00:03
   ---------- ----------------------------- 3.9/14.5 MB 3.2 MB/s eta 0:00:04
   -------------- ------------------------- 5.2/14.5 MB 3.8 MB/s eta 0:00:03
   ---------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-webui 0.6.6 requires fastapi==0.115.7, but you have fastapi 0.115.12 which is incompatible.
open-webui 0.6.6 requires pillow==11.1.0, but you have pillow 10.4.0 which is incompatible.


In [19]:
from langchain.vectorstores import Qdrant
import qdrant_client


d:\GBG\tasks\langchain\lang\Lib\site-packages\PIL\Image.py:122: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 10.4.0
Pillow version: 11.1.0
  warnings.warn(str(v), RuntimeWarning)
d:\GBG\tasks\langchain\lang\Lib\site-packages\PIL\Image.py:122: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 10.4.0
Pillow version: 11.1.0
  warnings.warn(str(v), RuntimeWarning)


In [ ]:
#qdrant client

os.environ["qdrant_host"] = ""
os.environ["qdrant_api_key"] = ""

qdrant_client = qdrant_client.QdrantClient(
    url=os.environ["qdrant_host"],
    api_key=os.environ["qdrant_api_key"],
)

d:\GBG\tasks\langchain\lang\Lib\site-packages\PIL\Image.py:122: RuntimeWarning: The _imaging extension was built for another version of Pillow or PIL:
Core version: 10.4.0
Pillow version: 11.1.0
  warnings.warn(str(v), RuntimeWarning)


In [21]:
#create collection
from qdrant_client import QdrantClient, models

os.environ["qdrant_collection_name"] = "my_collection"

vectors_config=models.VectorParams(size=1536,
                                    distance=models.Distance.COSINE)

qdrant_client.create_collection(
    collection_name=os.environ["qdrant_collection_name"],
    vectors_config=vectors_config,
    
)

True

In [24]:
#create vector store
from langchain.vectorstores import Qdrant

vector_store = Qdrant(
    client=qdrant_client,
    collection_name=os.environ["qdrant_collection_name"],
    embeddings=embedding_model,
)

In [25]:
#add documents to vector store
from langchain_core.documents import Document
from uuid import uuid4

all_langchain_documents = []

for s_doc in structured_docs:
    doc_name = s_doc['file']
    sections = s_doc['sections']

    for chunk_type, content in sections.items():
        if isinstance(content, list):
            for item in content:
                if item.strip(): 
                    all_langchain_documents.append(
                        Document(
                            page_content=item,
                            metadata={
                                "doc_name": doc_name,
                                "chunk_type": chunk_type,
                                "chunk_id": str(uuid4())
                            }
                        )
                    )
        elif isinstance(content, str):
            if content.strip():
                all_langchain_documents.append(
                    Document(
                        page_content=content,
                        metadata={
                            "doc_name": doc_name,
                            "chunk_type": chunk_type,
                            "chunk_id": str(uuid4())
                        }
                    )
                )

if all_langchain_documents:
    vector_store.add_documents(all_langchain_documents)